# Hand_Tracking with Mediapipe 
**Full Codiig available at [Hand_Gesture.py](Hand_gesture.py)**
<br>
This is the documantetion for Hand_Gesture.  

***Librarues Used In The Script*** 
- Picamera2 
- OpenCV 
- time 
- Motor_Encoder 
- PCA9685_MC 
- mediapipe 

## Installing Librarues@ Dependencies 
- Buuild in librarues 
    - Picamera2 
    - libcamera 
    - time 
- Used Installing Libraries
    - OpenCV 
    - Mediapipe 
- Cutomise Library 
    - Motor_Encoder 
    - PVA9685_MC 

### Install Libraries:
1.  OpenCV [OpenCV.org](https://docs.opencv.org/4.x/d2/de6/tutorial_py_setup_in_ubuntu.html)
    - In the terminal type `pip install opencv-python` 
    - In a Python File type: <br>
    `import cv2` <br>
    `print(cv2.__version__)`
    - The coding above will print the version of opencv library installed  
2. Mediapipe
    - In the terminal typr `pip install mediapipe`
    - In the same python file for OpenCV, Type:

        ```
        import mediapipe as mp 
        print(mp.__version__) 
        ```
    - The coding will print the version of the mediapipe library 

## Let's Start Coding ! 
### 1. Import the required libraries 
- OpenCV
- Mediapipe 
- Motor_Encoder
- PCA9685_MC
- picamera2



In [ ]:
import cv2
import mediapipe as mp
from picamera2 import Picamera2
from Motor_Encoder import Encoder 
from PCA9685_MC import Motor_Controller

### 2. The int function `def init()` 
- This function is used to initialise all the librariies and variables. 
- Using the `global` syntax, it enables the other funtion to get access to the internal variables in a function. <br>
`global mp_hands, hands, cap, mp_drawing, Motor, enc`
    - mp_hands: Mediapipe hands 
    - cap: the Camera 
    - mp_drawing: Mediapipe Dwawing 
    - Motor: To control the Motor 
    - enc: Motor Encoder
- All the configuration are also made in this fucntion.
    - Camera configuration :
        - `cap.configure(cap.create_preview_configuration(main={"format": 'XRGB8888', "size": (640, 480)}))` 
    - mediapipe configuration 
        - ```
            mp_hands = mp.solutions.hands
            hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1,
                                    min_detection_confidence=0.5, min_tracking_confidence=0.5)
            mp_drawing = mp.solutions.drawing_utils
          ```

In [ ]:
def init(): 
        """
        Initialize motor controller, encoder, mediapipe hands and camera 
        """
        global mp_hands, hands, cap, mp_drawing, Motor, enc
        Motor = Motor_Controller()
        enc = Encoder(ODISPLAY= True)
        # Initialize MediaPipe Hands
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1,
                                min_detection_confidence=0.5, min_tracking_confidence=0.5)
        mp_drawing = mp.solutions.drawing_utils

        # Start video capture
        cap = Picamera2(0)
        cap.configure(cap.create_preview_configuration(main={"format": 'XRGB8888', "size": (640, 480)}))
        cap.start()
       

### 3. The `control_car` fucntion 
- This function controls the car based on the hand's landmark 
- This function makes the robots to follow the 9th landmark of the palm. <br>
    <img src="https://www.researchgate.net/publication/357216549/figure/fig2/AS:1103448439304192@1640094017201/Hand-Landmark-in-MediaPipe-38.ppm" alt="Hand's landmarks position"> 
- The function will extract the landmark 9 informatio and get the x , y coordinates 
    ```
    landmark_9 = hand_landmarks.landmark[9]
    landmark_9_x = landmark_9.x
    landmark_9_y = landmark_9.y 
    ```

- Based on teh landmark position in the camera frame, This map the cordinates in to 6 condition which is:
    - Left 
    - Right 
    - Center 
    - Close 
    - Far 
    - Medium 
- Based on the conditions, the function will command the car to
    - Turn Left 
    - Turn Right 
    - Stop
    - Move forward 


In [ ]:
def control_car(hand_landmarks):
        global mp_hands, hands, cap, mp_drawing, Motor
        # Extract the position of landmark 9
        landmark_9 = hand_landmarks.landmark[9]
            
        # Get normalized coordinates of landmark 9
        landmark_9_x = landmark_9.x
        landmark_9_y = landmark_9.y
            
        # Calculate direction and distance based on landmark 9 position
        # Example: Determine if landmark 9 is to the left, right, or center
        motorFreq = 30 # set the speed of the motor to 20 
        if landmark_9_x < 0.3:
                direction = "left"
                
        elif landmark_9_x > 0.7:
                direction = "right"

        else:
                direction = "center"
            
        # Example: Determine if landmark 9 is closer or farther away
        if landmark_9_y < 0.4:
                distance = "close"
        elif landmark_9_y > 0.6:
                distance = "far"
        else:
                distance = "medium"
            
        # Map direction and distance to control commands for the car
        # Example: Adjust speed and steering based on direction and distance
        
        if direction == "left":
                print("Turn left")
                Motor.AntiClock_Rotate(15)
        elif direction == "right":
                print("Turn right")
                Motor.Clock_Rotate(15)
        else:
                # Move the car forward or stop based on distance
                if distance == "close":
                        print("Stop")
                        Motor.Brake()
                else:
                        print("Move forward")
                        Motor.Forward(motorFreq)

### 4. The `main` function 
- This function will run when the programe start. 
- This fucntion will handel all the image procesing 
- The **Whie True** Loop will start looping untill the programe being terminated or any keyboard interrupts. 
#### The `While True` Loop 
    - Obtain the frame capotured form the camera. 
    - convert the from from RGB(RED, GREEN. BLUE)  color format to BGR(BLUE, GREEN, RED) Color format    
